In [1]:
import boto3
import time
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

sagemaker_client = boto3.client("sagemaker", region_name="us-east-1")
s3_client = boto3.client("s3")

try:
    # Get AWS Account ID
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    print(f"Successfully retrieved AWS Account ID: {account_id}")
except Exception as e:
    print(f"Cannot retrieve account information: {e}")
    raise

Successfully retrieved AWS Account ID: 697347838118


In [3]:
print("Starting comprehensive cleanup process...")
print("=" * 80)

Starting comprehensive cleanup process...


In [9]:
# Delete Model Packages
print("\nDeleting Model Packages...")
try:
    model_package_group_name = f"venuesignal-model-group-{account_id}"

    list_packages = sagemaker_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        MaxResults=100
    )
    
    if list_packages["ModelPackageSummaryList"]:
        for pkg in list_packages["ModelPackageSummaryList"]:
            pkg_arn = pkg["ModelPackageArn"]
            try:
                sagemaker_client.delete_model_package(ModelPackageName=pkg_arn)
                print(f"   ✓ Model Package deleted: {pkg_arn}")
            except Exception as e:
                print(f"   ✗ Error deleting {pkg_arn}: {str(e)}")
        
        # Wait for deletion to complete
        time.sleep(5)
    else:
        print("   ℹ No model packages found")
except sagemaker_client.exceptions.ResourceNotFound:
    print(f"   ℹ Model Package Group not found")
except Exception as e:
    print(f"   ✗ Error listing model packages: {str(e)}")


Deleting Model Packages...
   ✓ Model Package deleted: arn:aws:sagemaker:us-east-1:697347838118:model-package/venuesignal-model-group-697347838118/2
   ✓ Model Package deleted: arn:aws:sagemaker:us-east-1:697347838118:model-package/venuesignal-model-group-697347838118/1


In [10]:
# Delete Model Package Group
print("\nDeleting Model Package Group...")
try:
    sagemaker_client.delete_model_package_group(
        ModelPackageGroupName=model_package_group_name
    )
    print(f"   ✓ Model Package Group '{model_package_group_name}' deleted")
except sagemaker_client.exceptions.ResourceNotFound:
    print(f"   ℹ Model Package Group not found (already deleted)")
except Exception as e:
    print(f"   ✗ Error deleting model package group: {str(e)}")


Deleting Model Package Group...
   ✓ Model Package Group 'venuesignal-model-group-697347838118' deleted


In [13]:
# Delete monitoring schedules
print("\nDeleting monitoring schedules...")
try:
    schedules = sagemaker_client.list_monitoring_schedules()
    for schedule in schedules['MonitoringScheduleSummaries']:
        schedule_name = schedule['MonitoringScheduleName']
        try:
            print(f"   Deleting Schedule: {schedule_name}")
            sagemaker_client.delete_monitoring_schedule(MonitoringScheduleName=schedule_name)
            print(f"   ✓ Schedule '{schedule_name}' deleted")
        except Exception as e:
            print(f"   ✗ Error deleting Schedule: {str(e)}")
        
        time.sleep(10)
    else:
        print("   ℹ No Schedules found")
except Exception as e:
    print(f"   ✗ Error listing Schedule: {str(e)}")



Deleting monitoring schedules...
   Deleting Schedule: xgboost-venuesignal-monitoring-schedule-697347838118
   ✓ Schedule 'xgboost-venuesignal-monitoring-schedule-697347838118' deleted
   ℹ No Schedules found


In [14]:
# Delete All Endpoints
print("\nDeleting Endpoints...")
try:
    endpoints = sagemaker_client.list_endpoints(MaxResults=100)
    
    if endpoints.get("Endpoints"):
        for endpoint in endpoints["Endpoints"]:
            endpoint_name = endpoint["EndpointName"]
            try:
                print(f"   Deleting Endpoint: {endpoint_name}")
                sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
                print(f"   ✓ Endpoint '{endpoint_name}' deleted")
            except Exception as e:
                print(f"   ✗ Error deleting endpoint: {str(e)}")
        
        # Wait for endpoints to be deleted
        time.sleep(10)
    else:
        print("   ℹ No endpoints found")
except Exception as e:
    print(f"   ✗ Error listing endpoints: {str(e)}")



Deleting Endpoints...
   Deleting Endpoint: venuesignal-endpoint-2026-02-10-01-19-21
   ✓ Endpoint 'venuesignal-endpoint-2026-02-10-01-19-21' deleted


In [12]:
# Delete Endpoint Configurations
print("\n Deleting Endpoint Configurations...")
try:
    endpoint_configs = sagemaker_client.list_endpoint_configs(MaxResults=100)
    
    if endpoint_configs.get("EndpointConfigs"):
        for config in endpoint_configs["EndpointConfigs"]:
            config_name = config["EndpointConfigName"]
            try:
                print(f"   Deleting Endpoint Config: {config_name}")
                sagemaker_client.delete_endpoint_config(EndpointConfigName=config_name)
                print(f"   ✓ Endpoint Config '{config_name}' deleted")
            except Exception as e:
                print(f"   ✗ Error deleting config: {str(e)}")
    else:
        print("   ℹ No endpoint configurations found")
except Exception as e:
    print(f"   ✗ Error listing endpoint configs: {str(e)}")


 Deleting Endpoint Configurations...
   Deleting Endpoint Config: venuesignal-endpoint-config-2026-02-10-01-18-51
   ✓ Endpoint Config 'venuesignal-endpoint-config-2026-02-10-01-18-51' deleted


In [15]:
# Delete Models
print("\nDeleting Models...")
try:
    models = sagemaker_client.list_models(MaxResults=100)
    
    if models.get("Models"):
        for model in models["Models"]:
            model_name = model["ModelName"]
            try:
                print(f"   Deleting Model: {model_name}")
                sagemaker_client.delete_model(ModelName=model_name)
                print(f"   ✓ Model '{model_name}' deleted")
            except Exception as e:
                print(f"   ✗ Error deleting model: {str(e)}")
    else:
        print("   ℹ No models found")
except Exception as e:
    print(f"   ✗ Error listing models: {str(e)}")


Deleting Models...
   Deleting Model: venuesignal-xgboost-2026-02-10-01-08-08-571
   ✓ Model 'venuesignal-xgboost-2026-02-10-01-08-08-571' deleted


In [4]:
bucket = f"yelp-aai540-group6-{account_id}"
# Delete S3 Objects
print("\nDeleting S3 Objects...")
try:
    print(f"   Bucket: {bucket}")
    
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket)
    
    delete_count = 0
    for page in pages:
        if 'Contents' in page:
            objects_to_delete = [{'Key': obj['Key']} for obj in page['Contents']]
            if objects_to_delete:
                s3_client.delete_objects(
                    Bucket=bucket,
                    Delete={'Objects': objects_to_delete}
                )
                delete_count += len(objects_to_delete)
                print(f"   Deleted {len(objects_to_delete)} objects...")
    
    print(f"   ✓ Total S3 objects deleted: {delete_count}")
except Exception as e:
    print(f"   ✗ Error deleting S3 objects: {str(e)}")


Deleting S3 Objects...
   Bucket: yelp-aai540-group6-697347838118
   ✓ Total S3 objects deleted: 0


In [10]:
# Athena
ATHENA_DB = "yelp"
ATHENA_RESULTS_PREFIX = "/athena/results/"
ATHENA_RESULTS_S3 = f"s3://{bucket}/athena/results/"
# Create PyAthena connection
conn = connect(
    s3_staging_dir=ATHENA_RESULTS_S3,
    region_name="us-east-1",
    cursor_class=PandasCursor
)

print(f" Connected to Athena database: {ATHENA_DB}")
print(f"   Results location: {ATHENA_RESULTS_S3}")

TABLES = ["business", "review", "user", "checkin", "tip", "business_attributes"]

for table in TABLES:
    query = f"DROP TABLE IF EXISTS {ATHENA_DB}.{table};"
    print(query)
    pd.read_sql(
        query,
        conn
    )

paginator = s3_client.get_paginator("list_objects_v2")
to_delete = []
for page in paginator.paginate(Bucket=bucket, Prefix=ATHENA_RESULTS_PREFIX):
    for obj in page.get("Contents", []):
        to_delete.append({"Key": obj["Key"]})

if not to_delete:
    print("✅ Nothing to delete under", f"s3://{bucket}{ATHENA_RESULTS_PREFIX}")
else:
    # delete in batches of 1000 (S3 limit)
    for i in range(0, len(to_delete), 1000):
        s3_client.delete_objects(
            Bucket=bucket,
            Delete={"Objects": to_delete[i:i+1000]}
        )
    print(f"✅ Deleted {len(to_delete)} objects under s3://{bucket}{ATHENA_RESULTS_PREFIX}")

print("✅ All tables dropped.")

 Connected to Athena database: yelp
   Results location: s3://yelp-aai540-group6-697347838118/athena/results/
DROP TABLE IF EXISTS yelp.business;
DROP TABLE IF EXISTS yelp.review;
DROP TABLE IF EXISTS yelp.user;
DROP TABLE IF EXISTS yelp.checkin;
DROP TABLE IF EXISTS yelp.tip;
DROP TABLE IF EXISTS yelp.business_attributes;
✅ Nothing to delete under s3://yelp-aai540-group6-697347838118/athena/results/
✅ All tables dropped.


In [2]:
# Delete Feature Store
print("\nDeleting Feature store...")
try:
    groups = sagemaker_client.list_feature_groups()
    print(groups)
    if groups.get("FeatureGroupSummaries"):
        for group in groups["FeatureGroupSummaries"]:
            group_name = group["FeatureGroupName"]
            try:
                print(f"   Deleting feature group: {group_name}")
                !aws sagemaker delete-feature-group --feature-group-name {group_name}
                print(f"   ✓ Feature Group '{group_name}' deleted")
            except Exception as e:
                print(f"   ✗ Error deleting Feature Group: {str(e)}")
    else:
        print("   ℹ No Feature Group found")
except Exception as e:
    print(f"   ✗ Error listing Feature Group: {str(e)}")


Deleting Feature store...
{'FeatureGroupSummaries': [{'FeatureGroupName': 'venuesignal-features-697347838118', 'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:697347838118:feature-group/venuesignal-features-697347838118', 'CreationTime': datetime.datetime(2026, 2, 9, 17, 18, 32, 830000, tzinfo=tzlocal()), 'FeatureGroupStatus': 'Created', 'OfflineStoreStatus': {'Status': 'Active'}}], 'ResponseMetadata': {'RequestId': '29b11569-886e-4677-b7be-089139e9e134', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '29b11569-886e-4677-b7be-089139e9e134', 'strict-transport-security': 'max-age=47304000; includeSubDomains', 'x-frame-options': 'DENY', 'content-security-policy': "frame-ancestors 'none'", 'cache-control': 'no-cache, no-store, must-revalidate', 'x-content-type-options': 'nosniff', 'content-type': 'application/x-amz-json-1.1', 'content-length': '296', 'date': 'Tue, 10 Feb 2026 03:39:01 GMT'}, 'RetryAttempts': 0}}
   Deleting feature group: venuesignal-features-697347838118
   ✓

In [ ]:
# Delete Feature Store
print("\nDeleting Feature store...")
try:
    groups = sagemaker_client.list_feature_groups()
    print(groups)
    if groups.get("FeatureGroupSummaries"):
        for group in groups["FeatureGroupSummaries"]:
            group_name = group["FeatureGroupName"]
            try:
                print(f"   Deleting feature group: {group_name}")
                !aws sagemaker delete-feature-group --feature-group-name {group_name}
                print(f"   ✓ Feature Group '{group_name}' deleted")
            except Exception as e:
                print(f"   ✗ Error deleting Feature Group: {str(e)}")
    else:
        print("   ℹ No Feature Group found")
except Exception as e:
    print(f"   ✗ Error listing Feature Group: {str(e)}")